# <font color="red">STEP 1: 文档提取测试</font>

In [33]:
# 初始化
import django_setup

In [34]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult

In [ ]:
# 准备测试所需的 user, project, file_record  (其中project与file_record关联)
User = get_user_model()

# 获取已存在的测试数据

# 获取已存在的用户
user = User.objects.get(phone='18501771516')
print(f"用户: {user.phone}")
        
# 获取已存在的项目
project = Project.objects.get(project_name='测试项目1')
print(f"项目: {project.project_name}")
        
# 获取已存在的文件
file_record = FileRecord.objects.get(id='3')
print(f"文件: {file_record.name}")

In [ ]:
# 清除‘测试分析A”，用于接下去的测试
DocumentAnalysis.objects.filter(title="测试分析A").delete()

In [ ]:
# 1. 创建新的文档分析实例 - 测试分析A
docx_analysis = DocumentAnalysis.objects.create(
    project=project,
    title="测试分析A",
    created_by=user,
    analysis_questions=["投标要求", "评分标准"]  # 示例分析问题
)
print(f"创建文档分析: {docx_analysis.title} (ID: {docx_analysis.id})")

In [ ]:
# 2.上传真实的 DOCX文件

# 2.1 准备文件路径
doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case8：招标文件-第1包：一级压榨花生油.docx"

# 2.2 读取文件内容
with open(doc_path, 'rb') as f:
    file_content = f.read()
test_file = SimpleUploadedFile(
    "test_doc.docx",
    file_content,
    content_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
)
print(f"文件大小: {test_file.size}")

# 2. 创建新的文件记录 并 存储文件对象
new_file_record = FileRecord.objects.create(
    name="test_doc.docx",
    file=test_file,  # 使用之前准备的测试文件
    owner=user,
    size = test_file.size
)
print(f"创建文件记录: {new_file_record.name}")

In [39]:
# 3. 关联新文件
try:
    docx_analysis.update_file_record(new_file_record)
except Exception as e:
    print(f"关联文件失败: {str(e)}")

In [ ]:
# 4. 触发开始分析，并提取文档元素 elements 存入数据库
print("\n===== 最终状态检查 =====")
print(f"开始分析前-状态: {docx_analysis.status}")
#docx_analysis.start_analysis()
print(f"开始分析后-状态: {docx_analysis.status}")

# 初始化DocxExtractorStep
docx_extractor=DocxExtractorStep()

try:
    # 准备输入数据, 好比 DocumentAnalysis.instance.data
    input_data = ModelData(model= DocumentAnalysis, instance=docx_analysis)
    
    # 执行文档提取
    docx_elements = docx_extractor.process(input_data)
    
    # 打印提取结果
    print("文档提取成功！提取到的元素数量:", len(docx_elements))
    print("第一个元素示例:", docx_elements[0])
    
    # 检查保存到数据库的结果
    saved_analysis = DocumentAnalysis.objects.get(id=docx_analysis.id)
    print("保存的提取结果:", saved_analysis.extracted_elements)
    
except ValidationError as e:
    print("文档提取失败:", str(e))
    # 检查分析状态
    failed_analysis = DocumentAnalysis.objects.get(id=docx_analysis.id)
    print("分析状态:", failed_analysis.status)
    print("错误信息:", failed_analysis.error_message)


# <font color="red">STEP 2: outline 分析测试</font>

In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from pprint import pprint 

In [ ]:
# 直接引用"测试分析A", 并获取其extracted_elements

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
extracted_elements = saved_analysis.extracted_elements
pprint(extracted_elements['elements'][10])

In [4]:
# 创建 DocxElements 实例 from models.py 的 extracted_elements
docx_elements = DocxElements.from_model(extracted_elements)

In [5]:
# 直接测试 outline_analyzer.py
outline_analyzer = DocxOutlineAnalyzerStep()


In [6]:
contexts, requirements, output_formats = outline_analyzer.prepare_requests_data(docx_elements)

In [ ]:
pprint(contexts)
pprint(requirements)
pprint(output_formats)


In [ ]:
analysis_result = outline_analyzer.process(docx_elements)

In [ ]:
# 打印输出的结果
print(type(analysis_result))
print(type(analysis_result.analysis_result))
print(type(analysis_result.analysis_result.result))
print(type(analysis_result.analysis_result.result[0]))
pprint(analysis_result)


In [ ]:
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
pprint(saved_analysis.outline_analysis_result)


# <font color="red">STEP 3: 测试outline_improvement.py</font>


In [4]:
# 初始化
import django_setup

In [5]:
# 导入相关模型
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from apps.doc_analysis.steps._03_outline_improvement import OutlineImprovementStep
from pprint import pprint 

In [6]:
# 直接引用"测试分析A", 并获取其docx_elements 和 outline_analysis_result

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
docx_elements = DocxElements.from_model(saved_analysis.extracted_elements)
outline_analysis_result = OutlineAnalysisResult.from_model(saved_analysis.outline_analysis_result)

In [ ]:
# 打印输出的结果
print(type(outline_analysis_result.analysis_result))
print(type(outline_analysis_result.analysis_result.result))
print(type(outline_analysis_result.analysis_result.result[0]))
pprint(outline_analysis_result)


## 分步骤测试

In [6]:
# 构建实例
outline_improver = OutlineImprovementStep()

In [ ]:
# titles 提取
titles_to_improve = outline_improver.extract_titles_from_analysis(outline_analysis_result)
pprint(titles_to_improve)


In [8]:
# 改进元素
improved_elements = outline_improver.improve_document_elements(docx_elements, titles_to_improve)

In [ ]:
# 打印改进后的数据
pprint(improved_elements)

## 集成测试

In [ ]:
# 模拟用户确认
outline_analysis_result.user_confirm = True
for element in outline_analysis_result.heading_only_elements:
    element['user_confirm'] = True
for element in outline_analysis_result.toc_only_elements:
    element['user_confirm'] = True

pprint(outline_analysis_result)


In [9]:
# 完整测试 _02_outline_analysis.py
outline_improver = OutlineImprovementStep()
improved_docx_elements = outline_improver.process((docx_elements, outline_analysis_result))

In [ ]:
# 打印原来的数据
pprint(docx_elements)

In [ ]:
# 打印改进后的数据
pprint(improved_docx_elements)

# <font color=red>STEP 4: 构建文档树DocxTree <font>


In [12]:
# 初始化
import django_setup

In [13]:
# show_structure函数（）
from pprint import pprint
from typing import Any

def show_structure(data: Any, name: str = "data", max_depth: int = None) -> None:
    """
    显示数据的类型和结构的辅助函数
    
    参数:
        data: 要检查的数据
        name: 数据的名称/标签
        max_depth: pprint的最大深度
    """
    print(f"\n{'='*50}")
    print(f"检查对象: {name}")
    print(f"类型: {type(data)}")
    
    if hasattr(data, '__len__'):
        print(f"长度: {len(data)}")
    
    print("\n结构:")
    pprint(data, depth=max_depth, width=100)
    print(f"{'='*50}\n")


In [14]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData, ImprovedDocxElements, SimpleDocxNode, DocxTree
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from pprint import pprint 

In [15]:
# 直接引用"测试分析A", 并获取 improved_docx_elements
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
improved_docx_elements = ImprovedDocxElements.from_model(saved_analysis.improved_docx_elements)
#elements = improved_docx_elements.elements
#show_structure(elements)


## 构建文档树

In [ ]:
# 构建文档树
build_docx_tree = BuildDocxTree()
docx_tree = build_docx_tree.process(improved_docx_elements)
show_structure(docx_tree)


In [ ]:
# 按顺序打印所有节点
pprint(docx_tree._ordered_nodes)

In [ ]:
# 按树结构打印所有节点
docx_tree.print_tree()

In [ ]:
# 检查文档树在models.py里的存储情况
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
print(type(saved_analysis.docxtree))
pprint(saved_analysis.docxtree)


## 测试从数据库提取文档树

In [ ]:
# 还原：从models.py存储到doc_tree类型的还原
docx_tree_from_model = DocxTree.from_model(saved_analysis.docxtree)
print(type(docx_tree_from_model))
pprint(docx_tree_from_model)

In [ ]:
# 还原 ： 按顺序打印所有节点
pprint(docx_tree_from_model._ordered_nodes)

In [ ]:
# 还原： 按树结构打印所有节点
docx_tree_from_model.print_tree()

## 测试文档树方法：查找节点，打印树，标题格式化

In [ ]:
# 查找某个节点 find_node()， 测试 .find_node() 方法，比如node_id = 78
node = docx_tree.find_node(78)
pprint(node)

In [ ]:
# 打印某个节点以下的文档树结构
docx_tree.print_tree(124)


In [ ]:
# 格式化 标题结构 用于大模型分析
titles_nodes = docx_tree.format_titles()
pprint(titles_nodes)

## 测试添加节点

In [ ]:
# 添加 节点
docx_tree.add_title_node(
    content = '>>>> 插入新节点',
    level = 2,
    after_node_id = 150 
)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

## 测试调整节点类型

In [19]:
# 节点改变层级
docx_tree.convert_to_title_node(156,2)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

# <font color=red>  STEP 5: 测试文档树的使用</font>

In [1]:
# 初始化
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-02-23 09:56:56,566 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData,DocxTree
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from pprint import pprint 

In [3]:
# 直接引用"测试分析A", 并获取 doc_tree
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
doc_tree = DocxTree.from_model(saved_analysis.docxtree)
print(type(doc_tree))
pprint(doc_tree)

<class 'apps.doc_analysis.pipeline.types.DocxTree'>
DocxTree(root=Node(0, 'ROOT...', [None]),
         document_analysis=ModelData(model=<class 'apps.doc_analysis.models.DocumentAnalysis'>,
                                     instance=<DocumentAnalysis: 测试分析A - 待分析>),
         _ordered_nodes=[Node(0, 'ROOT...', [None]),
                         Node(1, '北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项...', [paragraph]),
                         Node(2, '招标文件...', [paragraph]),
                         Node(3, '        项目名称：北京京铁运恒采购供应站有限公司 2...', [paragraph]),
                         Node(4, '项目编号：DLXM-2024-148-01...', [paragraph]),
                         Node(5, '招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中...', [paragraph]),
                         Node(6, '                2024 年 3 月...', [paragraph]),
                         Node(7, '                    目    录...', [toc]),
                         Node(8, '第一章  招标公告...', [toc]),
                         Node(9, '第二章  招标需求...', [toc]),
                        

In [4]:
llm_input = doc_tree.format_for_llm()
print(type(llm_input))
print(llm_input)


<class 'str'>
ROOT
    北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目
    招标文件
            项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油）
    项目编号：DLXM-2024-148-01
    招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司
                    2024 年 3 月
    【目录】:                     目    录
    【目录】: 第一章  招标公告
    【目录】: 第二章  招标需求
    【目录】: 第三章  投标人须知
    【目录】: 前附表
    【目录】: 一、总则
    【目录】: 二、招标文件
    【目录】: 三、投标文件的编制
    【目录】: 四、开标
    【目录】: 五、评标
    【目录】: 六、定标
    【目录】: 七、合同授予
    【目录】: 八、履约保证金
    【目录】: 九、招标代理服务费的收取
    【目录】: 第四章  评标办法及评分标准
    【目录】: 附件一：资格审查表
    【目录】: 附件二：符合性审查表
    【目录】: 附件三：评分标准
    【目录】: 第五章  合同条款及格式
    【目录】: 第六章  投标文件格式
    【目录】: 附件 1 投标函
    【目录】: 附件 2 投标保证书
    【目录】: 附件 3 无违法记录声明
    【目录】: 附件 4 法定代表人授权书
    【目录】: 附件 5 投标人情况简介
    【目录】: 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
    【目录】: 附件 7 投标一览表
    【目录】: 附件 8 分项报价表
    【目录】: 附件 9 投标方案
    【目录】: 附件 10 投标人资格声明
    【目录】: 附件 11 资格证明文件
    【目录】: 附件 12 与投标人存在“关系”的其他法人单位信息
    【目录】: 附件 13 投标人认为必要的其他证明文件
    【目录】: 附件 14 技术规格偏离表
    【目录】: 附件 

In [5]:
titles = doc_tree.format_titles()
print(type(titles))
print(titles)


<class 'str'>
    第一章  招标公告 [Level:1] [ID:46] [Tokens:2316]
    第二章 招标需求 [Level:1] [ID:95] [Tokens:1629]
    第三章  投标人须知 [Level:1] [ID:124] [Tokens:10019]
        前附表 [Level:2] [ID:125] [Tokens:2735]
        一、总则 [Level:2] [ID:130] [Tokens:1197]
        二、招标文件 [Level:2] [ID:159] [Tokens:635]
        三、投标文件的编制 [Level:2] [ID:175] [Tokens:3227]
        四、开标 [Level:2] [ID:256] [Tokens:379]
        五、评标 [Level:2] [ID:267] [Tokens:1191]
        六、定标 [Level:2] [ID:294] [Tokens:168]
        七、合同授予 [Level:2] [ID:298] [Tokens:82]
        八、履约保证金 [Level:2] [ID:300] [Tokens:54]
        九、招标代理服务费的收取 [Level:2] [ID:302] [Tokens:340]
    第四章  评标办法及评分标准 [Level:1] [ID:308] [Tokens:7039]
        附件一：资格审查表 [Level:2] [ID:352] [Tokens:1031]
        附件二：符合性审查表 [Level:2] [ID:355] [Tokens:498]
        附件三：评分标准 [Level:2] [ID:357] [Tokens:3406]
    第五章 合同条款及格式 [Level:1] [ID:366] [Tokens:7692]
        买卖合同 [Level:10] [ID:367] [Tokens:7682]
    第六章  投标文件格式 [Level:1] [ID:501] [Tokens:6844]
        附件 1 投标函 [Level:2]

In [ ]:
titles_json = doc_tree.titles_to_json()
print(type(titles_json))
pprint(titles_json)
